In [1]:
from functions import *

# ITU Data to TSV

In [2]:
itu_df = pd.read_csv(r"../data/public_data/train.csv", index_col="id")
itu_df.head()

,language,sentence,tokenized_sentence,expression,category,indices
id,,,,,,
1,tr,"Ödevler, Sınavlar, Evet yaşanması gereken zama...","['Ödevler', ',', 'Sınavlar', ',', 'Evet', 'yaş...",zaman öldürmek,idiomatic,"[7, 8]"
2,tr,Belediye bugün yolları açtı.,"['Belediye', 'bugün', 'yolları', 'açtı', '.']",yol açmak,literal,[-1]
3,tr,Planı ortaya çıkmasın diye elinden geleni yapmış.,"['Planı', 'ortaya', 'çıkmasın', 'diye', 'elind...",ortaya çıkmak,idiomatic,"[1, 2]"
4,tr,Aldığın bunca tedbire rağmen halen üstüne geli...,"['Aldığın', 'bunca', 'tedbire', 'rağmen', 'hal...",üstüne almak,literal,[-1]
5,tr,"Yazdıkların yanlış olmuş, istersen defterden s...","['Yazdıkların', 'yanlış', 'olmuş', ',', 'ister...",defterden silmek,literal,[-1]


In [3]:
train_path = r"../data/public_data/train.csv"
dev_path = r"../data/public_data/eval.csv"
test_path = r"../data/public_data/test.csv"

itu_to_tsv(train_path, r"./ITU/train.tsv")
itu_to_tsv(dev_path, r"./ITU/dev.tsv")

itu_to_tsv_test(test_path, r"./ITU/test.tsv")




# PARSEME Data to TSV

In [4]:
cupt_data = parse_cupt(r"../data/PARSEME 1.1 data/sharedtask-1.1-TR/train.cupt")
for word in cupt_data[1]:
    print(word)

('Türkiye', False)
("'de", False)
('bankaların', False)
('bir', False)
('siyasi', False)
('gücü', False)
('de', False)
('var', False)
(',', False)
('bu', False)
('nedenle', False)
('ilerde', False)
('örgütlenmeye', False)
('de', False)
('yardımcı', False)
('olabilirler', False)
('.', False)


In [5]:
# Ensure output folders exist
tr_out_path = r"../data/PARSEME 1.1 data/transformed-TR/"
it_out_path = r"../data/PARSEME 1.1 data/transformed-IT/"
os.makedirs(tr_out_path, exist_ok=True)
os.makedirs(it_out_path, exist_ok=True)

tr_in_path = r"../data/PARSEME 1.1 data/sharedtask-1.1-TR/"
it_in_path = r"../data/PARSEME 1.1 data/sharedtask-1.1-IT/"

# Transform Turkish
transform_cupt_to_tsv(tr_in_path+"train.cupt", tr_out_path+"train.tsv", "tr")
transform_cupt_to_tsv(tr_in_path+"dev.cupt", tr_out_path+"dev.tsv", "tr")
transform_cupt_to_tsv(tr_in_path+"test.cupt", tr_out_path+"test.tsv", "tr")

# Transform Italian
transform_cupt_to_tsv(it_in_path+"train.cupt", it_out_path+"train.tsv", "it")
transform_cupt_to_tsv(it_in_path+"dev.cupt", it_out_path+"dev.tsv", "it")
transform_cupt_to_tsv(it_in_path+"test.cupt", it_out_path+"test.tsv", "it")

In [6]:
# Create output folder
parseme_output_path = r"./PARSEME/"
os.makedirs(parseme_output_path, exist_ok=True)

# Combine train, dev, test TSVs
combine_tsv_files(tr_out_path+"train.tsv", it_out_path+"train.tsv", parseme_output_path+"train.tsv")
combine_tsv_files(tr_out_path+"dev.tsv", it_out_path+"dev.tsv", parseme_output_path+"dev.tsv")
combine_tsv_files(tr_out_path+"test.tsv", it_out_path+"test.tsv", parseme_output_path+"test.tsv")

# ID10M

ID10M is alredy in tsv format we need to add language.

In [7]:
id_input_path = r"../data/ID10M/"
# check outpath
out_path = r"./ID10M/"
os.makedirs(out_path, exist_ok=True)
# Add language to ID10M files
add_language(id_input_path+"train.tsv", out_path+"train.tsv", "it")
add_language(id_input_path+"dev.tsv", out_path+"dev.tsv", "it")
add_language(id_input_path+"test.tsv", out_path+"test.tsv", "it")

# combine all datasets

In [8]:
# Create output folder for combined dataset
combined_output_path = r"./COMBINED/"

os.makedirs(combined_output_path,exist_ok=True)

# Combine all train files into a single combined.tsv file
combine_all_tsv_files(r"./ITU/train.tsv",r"./PARSEME/train.tsv",r"./ID10M/train.tsv",combined_output_path+"train.tsv")
